In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

/home/ubuntu/varios/skforecast


In [15]:
from skforecast.recursive import ForecasterRecursive, ForecasterRecursiveMultiSeries
from skforecast.model_selection import TimeSeriesFold, backtesting_forecaster, backtesting_forecaster_multiseries
from lightgbm import LGBMRegressor
%load_ext pyinstrument
%load_ext line_profiler

The pyinstrument extension is already loaded. To reload it, use:
  %reload_ext pyinstrument
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [3]:
n = 10_000
y = pd.Series(
    np.random.normal(0, 1, n),
    index=pd.date_range('2020-01-01', periods=n, freq='h')
)
y

2020-01-01 00:00:00   -0.371184
2020-01-01 01:00:00   -0.634725
2020-01-01 02:00:00   -1.676311
2020-01-01 03:00:00    0.653251
2020-01-01 04:00:00   -1.433543
                         ...   
2021-02-20 11:00:00   -0.233228
2021-02-20 12:00:00   -0.579339
2021-02-20 13:00:00    0.150171
2021-02-20 14:00:00    1.322261
2021-02-20 15:00:00   -0.804803
Freq: h, Length: 10000, dtype: float64

In [4]:
forecaster = ForecasterRecursive(
    regressor=LGBMRegressor(verbose=-1),
    lags=48
)

In [5]:
%%timeit
forecaster.fit(y=y, store_in_sample_residuals=True)

303 ms ± 1.21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit
forecaster.fit(y=y, store_in_sample_residuals=False)

301 ms ± 2.47 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
cv = TimeSeriesFold(
    initial_train_size=7_000,
    steps=48,
)

In [8]:
%%timeit
_ = backtesting_forecaster(
    forecaster=forecaster,
    y=y,
    cv=cv,
    metric='mean_absolute_error',
    verbose=False,
    show_progress=False
)

2.58 s ± 3.76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit
_ = backtesting_forecaster(
    forecaster=forecaster,
    y=y,
    cv=cv,
    metric='mean_absolute_error',
    verbose=False,
    show_progress=False
)

2.58 s ± 3.84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
n = 1_000
n_series = 500
series = pd.DataFrame(
    np.random.normal(0, 1, n * n_series).reshape(n, n_series),
    index=pd.date_range('2020-01-01', periods=n, freq='h'),
    columns=[f'series_{i}' for i in range(n_series)]
)

In [6]:
forecaster = ForecasterRecursiveMultiSeries(
    regressor = LGBMRegressor(verbose=-1),
    lags     = 48
)

forecaster.fit(series=series, store_in_sample_residuals=False)

In [12]:
%%timeit
forecaster.fit(series=series)

6.46 s ± 19.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%timeit
forecaster.fit(series=series, store_in_sample_residuals=False)

5.54 s ± 30.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
cv = TimeSeriesFold(
     initial_train_size=int(len(series) * 0.8),
     steps = 24,
     refit = False
)

In [24]:
_ = backtesting_forecaster_multiseries(
    forecaster    = forecaster,
    series        = series,
    cv            = cv,
    metric        = "mean_absolute_error",
    show_progress = False,
    add_aggregated_metric = False
)

In [25]:
_ = backtesting_forecaster_multiseries(
    forecaster    = forecaster,
    series        = series,
    cv            = cv,
    metric        = "mean_absolute_error",
    interval             = [10, 90],
    n_boot = 10,
    use_binned_residuals = False,
    show_progress        = False
)

In [26]:
_ = backtesting_forecaster_multiseries(
    forecaster    = forecaster,
    series        = series,
    cv            = cv,
    metric        = "mean_absolute_error",
    interval             = [10, 90],
    n_boot = 10,
    use_binned_residuals = True,
    show_progress        = False
)

In [ ]:
21, 25, 54